In [ ]:
import numpy as np
from numpy import array as arr

import pandas
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl
from scipy.optimize import curve_fit
import uncertainties as unc
from uncertainties import unumpy as unp
from uncertainties.unumpy import uarray as uarr
from uncertainties.unumpy import nominal_values as val
from uncertainties.unumpy import std_devs as dev
from uncertainties import ufloat as uf

# import my plotting stuff

import sys
sys.path.append('./PythonHelpers/')
import PlotLib.Hist1D as Hist
import PlotLib.Plotting as Plot
from CSVimporter.importer import load_run
from CSVimporter.importer import dict_to_arr

def fitfunc_gauss(x, A, mu, sigma):
    return A * np.exp(-0.5*((x-mu)/sigma)**2)
def fitfunc_gauss_constBKG(x, A, mu, sigma, b):
    return A * np.exp(-0.5*((x-mu)/sigma)**2) + b
def fitfunc_doubleGauss(x, A, calibrationF, sigma):
    Ka_A = A
    Ka_mu = calibrationF
    Ka_sigma = sigma
    Kb_A = A / 24.4 * 2.85
    Kb_mu = calibrationF / 5.895 * 6.491
    Kb_sigma = sigma
    return Ka_A*np.exp(-0.5*((x-Ka_mu)/Ka_sigma)**2) + Kb_A*np.exp(-0.5*((x-Kb_mu)/Kb_sigma)**2)

In [ ]:
loadEvents = 100000
runIDs = arr([637,635,639,633,638,636,640,634])
# runIDs = arr([634])

runs = {}
for runID in runIDs:
    runs[runID] = load_run(runID, loadEvents*4, filepath="/home/jona/DESY/analysis_Fe55/output/csv/chargeCal_"+str(runID)+".csv")
    runs[runID]["nEvts"] = min(runs[runID]["nEvts"],loadEvents)
    
    runs[runID]["charge_cal"] = uarr([0]*4, [0]*4)
    runs[runID]["Ka_peakwidth"] = uarr([0]*4, [0]*4)
    runs[runID]["charge_cal_preCut"] = uarr([0]*4, [0]*4)

M = runs[runIDs[0]]["M"]
scalefactor = arr([1,100000/58975,1,1,1,1,1,1])

M.keys()
map_pixToAx = [2,0,3,1] # pix 00 -> bottom left -> ax[2]

In [ ]:
# 2d: Amplitude vs Risetime

runIDs = arr([637,635,639,633,638,636,640,634])

# rangeX = [5, 26]
rangeX = [5,20]
rangeY = [50, 185]
# binNX = 40
binNX = 40
binNY = 20
NameX = "Risetime"
NameY = "Amplitude"
logScale = False

risetime_cut_width = 0.212 * 3 # 1 sigma = 0.212 ns
risetime_regions = { # only keep this to show difference between the cuts done by eye (these here) and the automatic ones (that are used later one)
    637: [[6.,8],[6.,8],[6.,8],[6.,8]],
    635: [[6.,8.5],[6.,8.5],[6.,8.5],[6.,8.5]],
    639: [[5.5,7.5],[6.,8.5],[6.,8],[6.,8.5]],
    633: [[5.5,7.5],[6.,8],[5.5,7.5],[6.,8]],
    638: [[5.5,7.5],[5.5,8],[5.5,7.5],[5.5,8]],
    636: [[5.5,7],[5.5,8],[5.5,7.5],[5.5,8]],
    640: [[5.5,7],[5.5,8],[5.5,7.5],[5.5,8]],
    634: [[5.5,7],[5.5,7.7],[5.5,7.3],[5.5,7.7]],
}

pixMap = [1,3,0,2]
PixNames = arr(["Pix 0,0", "Pix 0,1", "Pix 1,0", "Pix 1,1"])

for runID in runIDs:
    runs[runID]["risetime_cut"] = np.zeros([4,2])
    
    FileName = f"Amplitude_vs_Risetime_{runs[runID]["krum_trim"]}DAC"
    fig, axs = Plot.create_fig(4,4, sharex=False, sharey=False, figsize=[6,4.5], width_ratios=[1,.2,1,.2], height_ratios=[.2,1,.2,1], flatten=False)        
    
    for i_row in range(2):
        for i_col in range(2):
            i_pix = i_row*2+i_col
            i_pix = pixMap[i_pix]
            mask = [True if x > 0 else False for x in runs[runID]["data"][:,i_pix,M[NameX]]]
            mask = np.logical_and(mask, np.count_nonzero(runs[runID]["data"][:,:,M["Charge"]], axis=1) < 2) # remove events with more than 1 pixel fired (decided 2025-03-11)
            entriesX = runs[runID]["data"][mask,i_pix,M[NameX]]
            entriesY = arr(runs[runID]["data"][mask,i_pix,M[NameY]])*1000
            hist, binsx, binsy = np.histogram2d(entriesX, entriesY, bins=[binNX, binNY], range=[rangeX, rangeY])
            
            # draw 2d histogram
            extent = [binsx[0], binsx[-1], binsy[0], binsy[-1]]
            axs[2*i_row+1, 2*i_col].imshow(hist.T, extent=extent, aspect="auto", origin='lower', cmap='viridis')
            
            # find highest bin, define cut region, draw vlines
            maxbin = np.unravel_index(np.argmax(hist, axis=None), hist.shape)
            maxbin_risetime = (binsx[maxbin[0]]+binsx[maxbin[0]+1])/2
            runs[runID]["risetime_cut"][i_pix] = arr([maxbin_risetime-risetime_cut_width, maxbin_risetime+risetime_cut_width])
            
            axs[2*i_row+1, 2*i_col].axvline(runs[runID]["risetime_cut"][i_pix,0], color="C1", lw=.5, ls="--")
            axs[2*i_row+1, 2*i_col].axvline(runs[runID]["risetime_cut"][i_pix,1], color="C1", lw=.5, ls="--")
            
            # axs[2*i_row+1, 2*i_col].axvline(risetime_regions[runID][i_pix][0], color="red", lw=.5, ls="--")
            # axs[2*i_row+1, 2*i_col].axvline(risetime_regions[runID][i_pix][1], color="red", lw=.5, ls="--")
        
            # draw y-projection
            hist1d, bins = np.histogram(entriesY, bins=binNY, range=rangeY)
            axs[2*i_row+1, 2*i_col+1].set_xlim(0, 1.3*np.max(hist1d))
            axs[2*i_row+1, 2*i_col+1].set_ylim(rangeY)
            axs[2*i_row+1, 2*i_col+1].stairs(hist1d, bins, color="black", orientation="horizontal", alpha=1, fill=False)
            axs[2*i_row+1, 2*i_col+1].stairs(hist1d, bins, color="black", orientation="horizontal",alpha=0.2, fill=True)
    
            # draw x-projection
            hist1d, bins = np.histogram(entriesX, bins=binNX, range=rangeX)
            axs[2*i_row, 2*i_col].set_xlim(rangeX)
            axs[2*i_row, 2*i_col].set_ylim(0, 1.3*np.max(hist1d))
            axs[2*i_row, 2*i_col].stairs(hist1d, bins, color="black", alpha=1, fill=False)
            axs[2*i_row, 2*i_col].stairs(hist1d, bins, color="black", alpha=0.2, fill=True)

            title_str = PixNames[i_pix]+"\nN={:.1f}k".format(np.sum(hist)/1000)
            axs[2*i_row+1, 2*i_col].text(0.5, 0.95, title_str, transform=axs[2*i_row+1, 2*i_col].transAxes, ha="center", fontdict={"color":"white"}, va="top")

    # Hist.finalize(runs[runID], fig, axs.flatten(), title=NameY+" vs "+NameX, xlabel=None, ylabel=None, subplots_adjust=[0.,0.,0.93])
    Plot.finalize(runs[runID], fig, axs.flatten(),
        NameY+" vs "+NameX + "\nrisetime cuts shown", None, None, None,
        title_linebreak=True, ER1=False,
        legend_loc=None, legend_fontdict={"size":"medium"}, legend_borderaxespad=3.5,
        param_dict={"ER1Param":True, "thr":None, "run":False, "sample":True, "fontsize":"medium"},
        grid=False, logy=False, subplots_adjust=None, labelpad=1.)
    fig.subplots_adjust(wspace=0., hspace=0)
    for ax in axs.flatten():
        ax.grid(False)
        ax.tick_params("both", direction="in", top=True, right=True)
    for i_row in range(3):
        for i_col in range(4):
            axs[i_row, i_col].set_xticklabels([])
            axs[i_col, i_row+1].set_yticklabels([])
    for i_row in range(2):
        for i_col in range(2):
            axs[2*i_row, 2*i_col].axis("off")
            axs[2*i_row, 2*i_col+1].axis("off")
            axs[2*i_row+1, 2*i_col+1].axis("off")
    for i in range(2):
        axs[2*i+1,0].set_ylabel("Amplitude / mV", loc="top")
        axs[-1, 2*i].set_xlabel("Risetime / ns", loc="right")
    
    Plot.savefig(fig, "ChargeCalibration/"+FileName)

In [ ]:
# krum_trim wise: Amplitude (& fit). without risetime cuts

binRange = arr([0.025,0.2])*1000
binN = 50
binWidth = (binRange[1]-binRange[0])/binN
key = "Amplitude Est."
runIDs = arr([637,635,639,633,638,636,640,634])
runIDs = arr([637,634])

binWidth = (binRange[1]-binRange[0])/binN

PixNames = arr(["Pix 0,0", "Pix 0,1", "Pix 1,0", "Pix 1,1"])
    
for i_pix in range(4):
    if len(runIDs) == 8:
        fig, axs = Plot.create_fig(2,4, [6,7.5])
    elif len(runIDs) == 2:
        fig, axs = Plot.create_fig(2,1, [6,3])
    Filename = f"Amplitude_krumTrimWise_Pix{i_pix}"
    
    for i,runID in enumerate(runIDs):
    
        # -- draw pre-cut amplitude --
        if True:    
            mask = np.count_nonzero(runs[runID]["data"][:,:,M["Charge"]], axis=1) < 2 # remove events with more than 1 pixel fired (decided 2025-03-11)
            entries = arr(runs[runID]["data"][mask,i_pix,M[key]])*1000
            hist, bins = np.histogram(entries, bins=binN, range=binRange)
                    
            label = "pre-cut"
            Plot.draw(axs[i], hist, bins, color="gray", label=label, fill_alpha=0.1, lw=1)
        
        # -- draw post-cut amplitude & fit -- 
        if True:
            runs[runID]["charge_cal"] = uarr([0]*4, [0]*4)
            # mask = (runs[runID]["data"][:,i_pix,M["Risetime"]] > risetime_regions[runID][i_pix][0]) & (runs[runID]["data"][:,i_pix,M["Risetime"]] < risetime_regions[runID][i_pix][1]) # old risetime cuts (done by eye)
            mask = (runs[runID]["data"][:,i_pix,M["Risetime"]] > runs[runID]["risetime_cut"][i_pix,0]) & (runs[runID]["data"][:,i_pix,M["Risetime"]] < runs[runID]["risetime_cut"][i_pix,1]) # automatic risetime cuts
            mask = np.logical_and(mask, np.count_nonzero(runs[runID]["data"][:,:,M["Charge"]], axis=1) < 2) # remove events with more than 1 pixel fired (decided 2025-03-11)

            entries = arr(runs[runID]["data"][mask,i_pix,M[key]])*1000
            hist, bins = np.histogram(entries, bins=binN, range=binRange)
            
            # find first bin left of the peak that is below 40% of peak amplitude. fit to everything right of that bin.
            max_bin = np.argmax(hist)
            cut_bin = max_bin - np.argmax(hist[:max_bin][::-1] < np.max(hist)*0.4) - 1
            fitMask = np.arange(0,len(hist)) > cut_bin
            
            p0 = arr([np.max(hist), 162, 5])
            pBounds = ([0, 100, 0], [10*np.max(hist), 200, 100])
            popt, perr, chi2, ndeg = Hist.curve_fit_wrapper(hist, bins, fitMask, fitfunc_doubleGauss, p0=p0, maxfev=100000, bounds=pBounds)
            
            runs[runID]["charge_cal"][i_pix] = 1617 / uf(popt[1], popt[2]) * 1000 # *1000 to convert back to V
            
            fitX = np.arange(bins[:-1][fitMask][0], bins[:-1][fitMask][-1]+binWidth, binWidth/20)
            x = np.arange(binRange[0], binRange[1],(binRange[1]-binRange[0])/1000)     

            axs[i].plot(fitX, fitfunc_doubleGauss(fitX,*popt), color="black", lw=.8)
            axs[i].plot(x, fitfunc_doubleGauss(x,*popt), color="black", lw=.5, ls=":", label="")
            
            label = "post-cut"
            Plot.draw(axs[i], hist, bins, color="C"+str(i_pix), label=label, fill_alpha=0.1)
            axs[i].set_xlim(binRange)
            
        axs[i].plot(0,0, color="black", lw=.8, label=r"$K_\alpha+K_\beta$ fit")
            
    axs[i].set_xlim(binRange)
    axs[0].set_ylim(0,axs[0].get_ylim()[1]*1.1)

    subtitles = dict_to_arr(runs, runIDs, "krum_trim")
    subtitles = ["Krummenacher\ntrimming = "+str(subtitles[i])+" DAC" for i in range(len(subtitles))]

    Plot.finalize(runs[runID], fig, axs,
        f"Waveform amplitude (pre/post risetime cut)\n{PixNames[i_pix]}", "Waveform amplitude / mV", f"Entries / {binWidth:.1f} mV", subtitles,
        title_linebreak=True, ER1=False,
        legend_loc=None, legend_fontdict={"size":"medium"}, legend_borderaxespad=3.5,
        param_dict={"ER1Param":False, "run":False, "sample":True, "fontsize":"medium"},
        grid=False, logy=False, subplots_adjust=None, labelpad=1.)
    axs[0].legend(frameon=False, loc="upper left", fontsize="small", borderaxespad=4)
    axs[0].set_xticks(axs[0].get_xticks()[1:])
    Plot.savefig(fig, "ChargeCalibration/"+Filename)

In [ ]:

# krum_trim wise: Amplitude (& fit). with risetime cuts

binRange = arr([0.025,0.2])*1000
binN = 50
binWidth = (binRange[1]-binRange[0])/binN
key = "AmplitudeEst"
runIDs = arr([637,635,639,633,638,636,640,634])

binWidth = (binRange[1]-binRange[0])/binN

# pixMap = [1,3,0,2]
PixNames = arr(["Pix 0,0", "Pix 0,1", "Pix 1,0", "Pix 1,1"])

for i_pix in range(4):
    Filename = f"Amplitude_krumTrimWise_postCut_Pix{i_pix}"
    print("\nPix",i_pix)
    print("DAC, Ka_mu[mV], Ka_sigma[mV], chi2red")
    
    fig, axs = Plot.create_fig(2,4, [6,7.5])
    for i,runID in enumerate(runIDs):
        # mask = (runs[runID]["data"][:,i_pix,M["Risetime"]] > risetime_regions[runID][i_pix][0]) & (runs[runID]["data"][:,i_pix,M["Risetime"]] < risetime_regions[runID][i_pix][1]) # old risetime cuts (done by eye)
        mask = (runs[runID]["data"][:,i_pix,M["Risetime"]] > runs[runID]["risetime_cut"][i_pix,0]) & (runs[runID]["data"][:,i_pix,M["Risetime"]] < runs[runID]["risetime_cut"][i_pix,1]) # automatic risetime cuts
        mask = np.logical_and(mask, np.count_nonzero(runs[runID]["data"][:,:,M["Charge"]], axis=1) < 2) # remove events with more than 1 pixel fired (decided 2025-03-11)

        entries = arr(runs[runID]["data"][mask,i_pix,M[key]])*1000
        hist, bins = np.histogram(entries, bins=binN, range=binRange)
        
        # find first bin left of the peak that is below 40% of peak amplitude. fit to everything right of that bin.
        max_bin = np.argmax(hist)
        cut_bin = max_bin - np.argmax(hist[:max_bin][::-1] < np.max(hist)*0.4) - 1
        fitMask = np.arange(0,len(hist)) > cut_bin
        
        p0 = arr([np.max(hist), 162, 5])
        pBounds = ([0, 100, 0], [10*np.max(hist), 200, 100])
        popt, perr, chi2, ndeg = Hist.curve_fit_wrapper(hist, bins, fitMask, fitfunc_doubleGauss, p0=p0, maxfev=100000, bounds=pBounds)
        
        runs[runID]["Ka_peakwidth"][i_pix] = uf(popt[2], perr[2]) / 1000
        runs[runID]["charge_cal"][i_pix] = 1617 / uf(popt[1], perr[1]) * 1000 # *1000 to convert back to V # e per V
        
        fitX = np.arange(bins[:-1][fitMask][0], bins[:-1][fitMask][-1]+binWidth, binWidth/20)
        x = np.arange(binRange[0], binRange[1],(binRange[1]-binRange[0])/1000)     

        axs[i].plot(fitX, fitfunc_doubleGaus160.0ust=None, labelpad=1.)
    
    axs[0].set_xticks(axs[0].get_xticks()[1:])
    Plot.savefig(fig, "ChargeCalibration/"+Filename)

In [ ]:
# (pix-wise) Fe55 peak position
Filename = "PeakPos_pixWise"
runIDs = arr([637,635,639,633,638,636,640,634])

fig, axs = Plot.create_fig(2,2,[6,4.5])
for i_pix in range(4):
    i_ax = map_pixToAx[i_pix]
    krum_bias_trimS = dict_to_arr(runs, runIDs, "krum_trim_nominal")
        
    charge_calS = arr([val(runs[runID]["charge_cal"][i_pix]) for runID in runIDs])
    charge_calS = 1000 / (charge_calS / 1617)
    charge_calS = uarr(charge_calS,[val(runs[runID]["Ka_peakwidth"][i_pix])*1000 for runID in runIDs])
    axs[i_ax].errorbar(krum_bias_trimS, val(charge_calS), yerr=dev(charge_calS), fmt="o-", color="C"+str(i_pix), label="", capsize=3, markersize=3, alpha=1)
    
    axs[i_ax].set_xticks(krum_bias_trimS, dict_to_arr(runs, runIDs, "krum_trim"))
    title = PixNames[i_pix]
    
axs[0].set_ylim(axs[0].get_ylim()[0],axs[0].get_ylim()[1]*1.05)
axs[0].legend(loc="lower left", fontsize="medium", borderaxespad=2, frameon=False)
    
subtitles = ["Pix 0,1", "Pix 1,1", "Pix 0,0", "Pix 1,0"]
Plot.finalize(runs[runID], fig, axs,
    r"$K_\alpha$ Peak Position and Width", "Krummenacher trimming [DAC]", r"$K_\alpha$ peak position [mV]", subtitles,
    title_linebreak=True, ER1=False,
    legend_loc=None, legend_fontdict={"size":"medium"}, legend_borderaxespad=2.5,
    param_dict={"ER1Param":False, "run":False, "sample":True, "fontsize":"medium"},
    grid=False, logy=False, subplots_adjust=None, labelpad=1.)
Plot.savefig(fig, "ChargeCalibration/"+Filename)

In [ ]:
# (pix-wise) Calibration factors
Filename = "Gain_pixWise"
runIDs = arr([637,635,639,633,638,636,640,634])

fig, axs = Plot.create_fig(2,2,[6,4.5])
for i_pix in range(4):
    i_ax = map_pixToAx[i_pix]
    krum_bias_trimS = dict_to_arr(runs, runIDs, "krum_trim_nominal")
        
    charge_calS = uarr([val(runs[runID]["charge_cal"][i_pix]) for runID in runIDs],[dev(runs[runID]["charge_cal"][i_pix]) for runID in runIDs])
    # charge_cal = e / V
    charge_calS = 1 / (charge_calS/1000)
    axs[i_ax].errorbar(krum_bias_trimS, val(charge_calS), yerr=dev(charge_calS), fmt="o-", color="C"+str(i_pix), label="", capsize=3, markersize=3, alpha=1)
    
    axs[i_ax].set_xticks(krum_bias_trimS, dict_to_arr(runs, runIDs, "krum_trim"))
    title = PixNames[i_pix]
    
# axs[0].set_ylim(9300,11100)
axs[0].legend(loc="lower left", fontsize="medium", borderaxespad=2, frameon=False)
    
subtitles = ["Pix 0,1", "Pix 1,1", "Pix 0,0", "Pix 1,0"]
Plot.finalize(runs[runID], fig, axs,
    r"Gain (mV / $e$)", "Krummenacher trimming [DAC]", r"Charge calibration factor [mV/$e$]", subtitles,
    title_linebreak=True, ER1=False,
    legend_loc=None, legend_fontdict={"size":"medium"}, legend_borderaxespad=2.5,
    param_dict={"ER1Param":False, "run":False, "sample":True, "fontsize":"medium"},
    grid=False, logy=False, subplots_adjust=None, labelpad=1.)
Plot.savefig(fig, "ChargeCalibration/"+Filename)

In [ ]:
# Gain
Filename = "Gain"
runIDs = arr([637,635,639,633,638,636,640,634])

fig, ax = Plot.create_fig(1,1,[4.5,3])
for i_pix in range(4):
    krum_bias_trimS = dict_to_arr(runs, runIDs, "krum_trim_nominal")
        
    charge_calS = uarr([val(runs[runID]["charge_cal"][i_pix]) for runID in runIDs],[dev(runs[runID]["charge_cal"][i_pix]) for runID in runIDs])
    # charge_cal = e / V
    charge_calS = 1 / (charge_calS/1000)
    ax.errorbar(krum_bias_trimS, val(charge_calS), yerr=dev(charge_calS), fmt="o-", color="C"+str(i_pix), label=PixNames[i_pix], capsize=3, markersize=3, alpha=1)
    
    ax.set_xticks(krum_bias_trimS, dict_to_arr(runs, runIDs, "krum_trim"))
    # title = PixNames[i_pix]
    
# axs[0].set_ylim(9300,11100)
# axs[0].legend(loc="lower left", fontsize="medium", borderaxespad=2, frameon=False)
    
subtitles = ["Pix 0,1", "Pix 1,1", "Pix 0,0", "Pix 1,0"]
Plot.finalize(runs[runID], fig, ax,
    r"Gain (mV / $e$)", "Krummenacher trimming [DAC]", r"Charge calibration factor [mV/$e$]", subtitles,
    title_linebreak=True, ER1=False,
    legend_loc="upper right", legend_fontdict={"size":"medium"}, legend_borderaxespad=1,
    param_dict={"ER1Param":False, "run":False, "sample":True, "fontsize":"medium"},
    grid=False, logy=False, subplots_adjust=None, labelpad=1)
Plot.savefig(fig, "ChargeCalibration/"+Filename)

In [ ]:
def get_calibration_factors(runs_, krum_trims):
    if len(krum_trims) != 4:
        print("Error: krum_trims must have length 4, coresponding to the 4 pixels")\
    
    cal_runIDs = arr([637,635,639,633,638,636,640,634])
    cal_krum_trims = dict_to_arr(runs_, cal_runIDs, "krum_trim")
    
    cal_factors = np.zeros(4)
    for i_pix in range(4):
        cal_runID = cal_runIDs[np.where(krum_trims[i_pix] == cal_krum_trims)[0][0]]
        cal_factors[i_pix] = runs_[cal_runID]["charge_cal"][i_pix].nominal_value
    return cal_factors

# load TB runs
loadEvents = 10
runIDs = arr([184,190,192])

if not "runs" in globals():
    runs = {}
    
for runID in runIDs:
    runs[runID] = load_run(runID, loadEvents*4, filepath=f"/home/jona/DESY/analysis_TB/output/csv/good/{runID}_analysis.csv")
    runs[runID]["nEvts"] = min(runs[runID]["nEvts"],loadEvents)    

# get calibraton factors
for runID in runIDs:
    print(str(runID)+": ", end="")
    krum_trims = runs[runID]["krum_trim"]
    
    factors = get_calibration_factors(runs, krum_trims)
    for factor in factors[:-1]:
        print("{:.2f}".format(factor), end=",")
    print("{:.2f}".format(factors[-1]), end="\n")